# create an agent with adk, deploy it to Agent Engine and publish it to Agentspace

# Authentication
Mostly for Google Colab.  
You will be authenticated with your user in Colab Enterprise (Vertex AI)

In [ ]:
import sys

if "google.colab" in sys.modules:
    print("Running in Google Colab environment")
    from google.colab import auth
    auth.authenticate_user()

# Configuration

In [34]:
# @title Settings
#@markdown ---
#@markdown ### Configure your settings below:

#@markdown _What is the ID of your Google Cloud project._

PROJECT_ID = "dbr-console-testing-1xziqpsq" #@param {type:"string"}
#@markdown _The Google Cloud region you are using_
LOCATION = "us-central1"  #@param {type:"string"}
#@markdown _The model to use is_
MODEL = "gemini-2.0-flash-001" #@param ["gemini-2.0-pro-exp-02-05","gemini-1.5-pro-002","gemini-1.5-flash-001","gemini-2.0-flash-001"]
#@markdown _The name of your agent in Agent Engine_
AE_APP_NAME = "My agent in A-Engine"  #@param {type:"string"}
#@markdown _The ID of your existing Agentspace application_
AS_APP="dom-as_1753424990568" #@param {type:"string"}

import os
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["AE_APP_NAME"] = AE_APP_NAME
os.environ["AS_APP"] = AS_APP


In [ ]:
!pip install --upgrade "google-cloud-aiplatform[adk,agent_engines]>1.100"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Configure Vertex AI

In [ ]:
import vertexai
from dotenv import load_dotenv

load_dotenv()
vertexai.init(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket="gs://" + os.getenv("GOOGLE_CLOUD_PROJECT")+"-bucket",
)

# Agent definition
- no external dependencies
- using a tool avalable in ADK

In [ ]:
from google.adk import Agent
from google.adk.tools import google_search  # The Google Search tool

root_agent = Agent(
    # name: A unique name for the agent.
    name="google_search_agent",
    # description: A short description of the agent's purpose, so
    # other agents in a multi-agent system know when to call it.
    description="Answer questions using Google Search.",
    # model: The LLM model that the agent will use:
    model="gemini-2.0-flash-001",
    # instruction: Instructions (or the prompt) for the agent.
    instruction="You are an expert researcher. You stick to the facts.",

    # tools: functions to enhance the model's capabilities.
    # Add the google_search tool below.
    tools=[google_search]
)

# Running locally
- The agent will run in the Notebook

In [ ]:
# @title User input for the local test {"run":"auto","vertical-output":true}
question = "what is the capital of France?" #@param {type:"string"}

In [ ]:
from vertexai.preview.reasoning_engines import AdkApp
from IPython.display import Markdown

# Create a simple ADK app for the root_agent.
my_agent =AdkApp(
    agent=root_agent,
    enable_tracing=True
)

# We need a session to use the agent
local_session = my_agent.create_session(user_id="user1")

# Stream query to get the answer
# Naively print the text of the 1st part of the answer (assuming it exists)
for event in my_agent.stream_query(
    user_id="user1",
    session_id=local_session.id,
    message=question,
):
   # not robust way to print the result! should test for each element and aggregate parts into 1 answer
   print("[local test] " + event["content"]["parts"][0]["text"])


# Deploy to Agent Engine
This will  
1) pack agent and dependencies to a GCS bucket  
2) Create the agent engine a.k.a. a reasoning engine  
3)Deploy the agent to Agent engine

In [ ]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    display_name=os.getenv("AE_APP_NAME", "my agent App"),
    agent_engine=root_agent,
    extra_packages=[],
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ]
)

# Publish to Agentspace
Assumption: you have an existing Agentspace app available  


In [37]:
%%bash
export REASONING_ENGINE_NUMBER=3396655300990730240
echo "projects/${GOOGLE_CLOUD_PROJECT}/locations/${GOOGLE_CLOUD_LOCATION}/reasoningEngines/${REASONING_ENGINE_NUMBER}"


projects/dbr-console-testing-1xziqpsq/locations/us-central1/reasoningEngines/3396655300990730240


In [ ]:
%%bash
# Agentspace
export REASONING_ENGINE_NUMBER=3396655300990730240
# the agent you want to register in Agent space - MUST be in the same project
export AGENT_ENGINE="projects/${GOOGLE_CLOUD_PROJECT}/locations/${GOOGLE_CLOUD_LOCATION}/reasoningEngines/${REASONING_ENGINE_NUMBER}"

# for Agentspace
export DISPLAY_NAME="a simple Google search agent"
export DESCRIPTION="This agent is using the google_search tool to help the user"

# where you register your Agent
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/projects/${GOOGLE_CLOUD_PROJECT}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents"

# --- Register Agent 1: Meeting Assistant Agent ---
echo "Registering Agent: ${DISPLAY_NAME}..."
curl -s -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \
-d @-<<EOF
{
  "displayName": "${DISPLAY_NAME}",
  "description": "${DESCRIPTION}",
  "adk_agent_definition": {
    "tool_settings": {
      "tool_description": "Use gogle_search",
    },
    "provisioned_reasoning_engine": {
      "reasoning_engine": "${AGENT_ENGINE}"
    }
  }
}
EOF
echo -e "\nAgent  registration attempt finished.\n"